# Extraindo Dados
___

Esse arquivo tem o objetivo de extrair dados do dataframe BSI e criar uma tabela que contenha os seguintes dados:

- o identificador do discente; 
- o sexo do discente;
- o ano em que o discente ingressou na instituição;
- o ano e período da última matrícula realizada pelo discente;
- total de sementres cursados pelo discente;
- um calcúlo de semestres cursados dividido pelo o número de semestre ideal do curso;
- a carga horária cumprida pelo discente;
- a carga horária cumprida pelo discente dividida pela carga horaria ideal;
- tempo relativo que o discente evadiu ou concluiu o curso;
- status atual do discente;
- número total de faltas de cada discente na disciplina que ele cursou;
- nota de cada discente nas disciplinas que ele cursou;
- média final de cada discente nas disciplinas que ele cursou;
- quantidade de vezes que o discente se matriculou nas disciplinas;
- quantidade de vezes que o discente se matriculou em cada disciplina;
- quantidade de vezes que o discente foi aprovado em cada disciplina;
- quantidade de vezes que o discente foi reprovado em cada disciplina.

Importando o Pandas e o Csv.

In [4160]:
import pandas as pd
import numpy as np
import csv

Leitura do arquivo em csv e carregamento desses dados em um DataFrame usando o ponto e vírgula como separador.

In [4161]:
df_dados = pd.read_csv('dataframe-bsi-2009-2022.csv', sep=';')

In [4162]:
pd.set_option('display.max_columns', 130)

# Tratamento de Dados

Listando as colunas do dataframe.

In [4163]:
df_dados.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'frequencia', 'descricao',
       'ano', 'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status'],
      dtype='object')

Transformando a coluna *media_final* no tipo float.

In [4164]:
df_dados['media_final']=df_dados['media_final'].apply(lambda x: str(x).replace(',' , '.'))
df_dados['media_final'] = df_dados['media_final'].astype('float64')

Transformando a coluna *nota* no tipo float.

In [4165]:
df_dados['nota']=df_dados['nota'].apply(lambda x: str(x).replace(',' , '.'))
df_dados['nota'] = df_dados['nota'].astype('float64')

Informações sobre o tipo de cada coluna do dataframe.

In [4166]:
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54584 entries, 0 to 54583
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   discente        54584 non-null  object 
 1   unidade         50972 non-null  float64
 2   nota            48321 non-null  float64
 3   media_final     47687 non-null  float64
 4   frequencia      50471 non-null  float64
 5   descricao       54584 non-null  object 
 6   ano             54584 non-null  int64  
 7   id_componente   54584 non-null  int64  
 8   nome            54584 non-null  object 
 9   ch_total        54584 non-null  int64  
 10  sexo            54584 non-null  int64  
 11  ano_nascimento  54584 non-null  int64  
 12  ano_ingresso    54584 non-null  int64  
 13  status          54584 non-null  object 
dtypes: float64(4), int64(6), object(4)
memory usage: 5.8+ MB


Listando os dados NaN.

In [4167]:
df_dados.isnull().sum()

discente             0
unidade           3612
nota              6263
media_final       6897
frequencia        4113
descricao            0
ano                  0
id_componente        0
nome                 0
ch_total             0
sexo                 0
ano_nascimento       0
ano_ingresso         0
status               0
dtype: int64

Preenchendo os dados NaN da coluna *unidade* com 1.

In [4168]:
df_dados.loc[:, 'unidade'] = df_dados['unidade'].fillna(1)

Substituir valores 0 por -1 nas colunas *nota*, *media_final* e *frequencia*.

In [4169]:
df_dados[['nota', 'media_final']] = df_dados[['nota', 'media_final']].replace(0, -1)

Apagando da coluna *descricao* a condição *INDEFERIDO* e *EXCLUIDA*.

In [4170]:
df_dados = df_dados[~df_dados['descricao'].isin(['INDEFERIDO', 'EXCLUIDA'])]

Verificar os valores faltosos na coluna *nota* com base na condição da coluna *descricao*.

In [4171]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'nota'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'nota'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'nota'] = -7

Verificar os valores faltosos na coluna *media_final* com base na condição da coluna *descricao*.

In [4172]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'media_final'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'media_final'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'media_final'] = -7

Verificar os valores faltosos na coluna *frequencia* com base na condição da coluna *descricao*.

In [4173]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'frequencia'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'frequencia'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'frequencia'] = -7

Contando os valores dos dados da coluna *descricao*.

In [4174]:
df_dados.descricao.value_counts()

descricao
APROVADO                            28307
APROVADO POR NOTA                    8178
REPROVADO                            5790
REPROVADO POR MÉDIA E POR FALTAS     3804
CANCELADO                            2730
TRANCADO                             1913
DESISTENCIA                           457
REPROVADO POR FALTAS                  138
REPROVADO POR NOTA                     99
REPROVADO POR NOTA E FALTA             42
Name: count, dtype: int64

Criar uma função para mapear a classificação da coluna *descricao*.

In [4175]:
def mapear_classificacao(descricao):
    mapeamento = {
        'APROVADO': 2,
        'APROVADO POR NOTA': 1,
        'REPROVADO': -1,
        'REPROVADO POR FALTAS': -1,
        'REPROVADO POR NOTA': -1,
        'REPROVADO POR MÉDIA E POR FALTAS': -2,
        'REPROVADO POR NOTA E FALTA': -2,
        'CANCELADO': -3,
        'EXCLUIDA': -3,
        'TRANCADO': -3,
        'DESISTENCIA': -4,
    }
    return mapeamento.get(descricao, None)

Aplicar a função de mapeamento e criar a coluna *classificacao*.

In [4176]:
df_dados.loc[:, 'classificacao'] = df_dados['descricao'].map(mapear_classificacao)

Fazendo um recorte da nossa análise, vamos começar por disciplinas obrigatórias do Bacharelado em Sistemas da Informação (BSI):

In [4177]:
lista_obrigatórias = [
                'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                'INTRODUÇÃO À INFORMÁTICA',
                'FUNDAMENTOS DE MATEMÁTICA',
                'LÓGICA',
                'TEORIA GERAL DA ADMINISTRAÇÃO',
                'PROGRAMAÇÃO',
                'CÁLCULO DIFERENCIAL E INTEGRAL',
                'TEORIA GERAL DOS SISTEMAS',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                'ESTRUTURA DE DADOS',
                'ÁLGEBRA LINEAR',
                'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                'PROGRAMAÇÃO WEB',
                'ARQUITETURA DE COMPUTADORES',
                'PROBABILIDADE E ESTATÍSTICA',
                'BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE I',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                'SISTEMAS OPERACIONAIS',
                'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE II',
                'REDES DE COMPUTADORES',
                'EMPREENDEDORISMO EM INFORMÁTICA',
                'GESTÃO DE PROJETO DE SOFTWARE',
                'PROGRAMAÇÃO VISUAL',
                'MATEMÁTICA FINANCEIRA',
                'SISTEMAS DE APOIO À DECISÃO',
                'ÉTICA',
                ]
condição_nome = f"nome in {lista_obrigatórias}"
df_dados_filtrado = df_dados.query(condição_nome)

## Extraindo dados

In [4178]:
df_dados_filtrado.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'frequencia', 'descricao',
       'ano', 'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status', 'classificacao'],
      dtype='object')

### Colunas com as **Disciplinas**

Filtrar os dados onde a coluna **unidade** foi preenchida com 1.

In [4179]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Calculando a quantidade de vezes que cada discente cursou cada *disciplina*.

In [4180]:
quantidade_disciplinas = filtrando_unidade.groupby(['discente', 'nome']).size().reset_index(name='quantidade')


Pivotando as *disciplinas*.

In [4181]:
tabela_final = quantidade_disciplinas.pivot(index='discente', columns='nome', values='quantidade').reset_index()

In [4182]:
tabela_final.replace(0, -1, inplace=True)

Substituir NaN por 0 usando fillna.

In [4183]:
tabela_final = tabela_final.fillna(0)

### Coluna com o **Ano de Ingresso**

Adicionando o *ano_ingresso* para cada discente.

In [4184]:
ano_ingresso_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'ano_ingresso']]
tabela_final = tabela_final.merge(ano_ingresso_discente, on='discente', how='left')

### Coluna com o **Sexo** do Discente

Definir a opção global para permitir o downcasting silencioso

In [4185]:
pd.set_option('future.no_silent_downcasting', True)

Adicionar as colunas *mulher* e *homem* usando o método assign para evitar SettingWithCopyWarning.

In [4186]:
df_dados_filtrado = df_dados_filtrado.assign(
    mulher=df_dados_filtrado['sexo'].map({0: 0, 1: 1}),
    homem=df_dados_filtrado['sexo'].map({0: 1, 1: 0})
)

Criar DataFrames com discente e as novas colunas.

In [4187]:
sexo_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'mulher', 'homem']]

Mesclar com tabela_final.

In [4188]:
tabela_final = tabela_final.merge(sexo_discente, on='discente', how='left')


### Coluna com o **Status** do Discente

Alterando o conteúdo da coluna *status*.

In [4189]:
df_dados_filtrado['status'] = df_dados_filtrado['status'].replace({
    'CANCELADO': -1, 
    'ATIVO': 0, 'ATIVO - FORMANDO': 0,
    'FORMADO': 1,'CONCLUÍDO': 1})

Adicionando o *status* para cada discente.


In [4190]:
status_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'status']]
tabela_final = tabela_final.merge(status_discente, on='discente', how='left')

### Coluna com o **classificacao** da Descrição

Adicionando o *classificacao* para cada discente.

In [4191]:
classificacao_descricao = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'classificacao']]
tabela_final = tabela_final.merge(classificacao_descricao, on='discente', how='left')

### Coluna com a **Carga Horária Cumprida**

In [4192]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Agrupando por discente e somando a *carga horária*.

In [4193]:
df_carga_horaria_cumprida = filtrando_unidade.groupby('discente')['ch_total'].sum().reset_index()

Adicionando a *carga horária cumprida* de cada discente e renomeando a coluna.

In [4194]:
df_carga_horaria_cumprida.rename(columns={'ch_total': 'ch_cumprida'}, inplace=True)
tabela_final = tabela_final.merge(df_carga_horaria_cumprida, on='discente', how='left')

Dividindo a *carga horária cumprida* do discente pela carga horária das disciplinas obrigatórias.

In [4195]:
tabela_final['ch_cumprida_dividida'] = tabela_final['ch_cumprida'] / 1830

### Coluna com a **Carga Horária Cumprida** com **Reprovação**

Filtrando dados pela unidade.

In [4196]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Filtrar as disciplinas *reprovadas*.

In [4197]:
reprovadas_status = [
    'REPROVADO',
    'REPROVADO POR NOTA',
    'REPROVADO POR FALTAS',
    'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR MÉDIA E POR FALTAS'
]

In [4198]:
df_reprovadas = filtrando_unidade[filtrando_unidade['descricao'].isin(reprovadas_status)]

Agrupar por discente e somar a *carga horária das disciplinas reprovadas*.

In [4199]:
ch_reprovacao = df_reprovadas.groupby('discente')['ch_total'].sum().reset_index()

Renomear a coluna resultante.

In [4200]:
ch_reprovacao.columns = ['discente', 'ch_reprovacao']

Adicionando a *carga horária de reprovação* de cada discente.

In [4201]:
tabela_final = tabela_final.merge(ch_reprovacao, on='discente', how='left')
df_dados_filtrado = df_dados_filtrado.merge(ch_reprovacao, on='discente', how='left')
df_dados_filtrado

,discente,unidade,nota,media_final,frequencia,descricao,ano,id_componente,nome,ch_total,sexo,ano_nascimento,ano_ingresso,status,classificacao,mulher,homem,ch_reprovacao
0,bad4e0ec61e635f382e7e32c32d5ad11,1.0,-1.0,-1.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0
1,bad4e0ec61e635f382e7e32c32d5ad11,2.0,-1.0,-1.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0
2,bad4e0ec61e635f382e7e32c32d5ad11,3.0,-1.0,-1.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0
3,bee926721a5b23a098fdfb850bf8d5b5,1.0,-1.0,-1.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1,0,1,330.0
4,bee926721a5b23a098fdfb850bf8d5b5,2.0,-1.0,-1.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1,0,1,330.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38085,22f4aed4a073c5e9515a8669e9c102f3,2.0,6.9,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1,0,1,240.0
38086,22f4aed4a073c5e9515a8669e9c102f3,3.0,5.0,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1,0,1,240.0
38087,943463d2de8c6a60ec5f5d959ba7f1ac,1.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2,0,1,60.0
38088,943463d2de8c6a60ec5f5d959ba7f1ac,2.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2,0,1,60.0


### Coluna com **taxa de reprovação**

In [4202]:
tabela_final['taxa_reprovacao'] = tabela_final['ch_reprovacao'] / tabela_final['ch_cumprida'] 

### Coluna com o **Semestre**

Função para calcular o *semestre* com base no *ano de ingresso* e o *ano cursado.*

In [4203]:
def calcular_semestre(ano_ingresso, ano_cursado):
    try:
        # Convertendo os anos para strings e ajustando para o formato correto
        ano_ingresso_str = str(ano_ingresso) + '1' if len(str(ano_ingresso)) == 4 else str(ano_ingresso)
        ano_cursado_str = str(ano_cursado) + '1' if len(str(ano_cursado)) == 4 else str(ano_cursado)
        
        ano_ingresso_num = int(ano_ingresso_str[:4])
        semestre_ingresso = int(ano_ingresso_str[4])
        
        ano_cursado_num = int(ano_cursado_str[:4])
        semestre_cursado = int(ano_cursado_str[4])
        
        anos_passados = (ano_cursado_num - ano_ingresso_num) * 2
        semestres_passados = anos_passados + (semestre_cursado - semestre_ingresso)
        
        return semestres_passados + 1  # +1 porque o semestre de ingresso é contado como o primeiro
    except (IndexError, ValueError):
        return pd.NA  # Retorna NA se houver um problema com o formato dos anos

Corrigir os valores das colunas *ano_ingresso* e *ano*.

In [4204]:
df_dados_filtrado['ano_ingresso'] = df_dados_filtrado['ano_ingresso'].astype(str)
df_dados_filtrado['ano'] = df_dados_filtrado['ano'].astype(str)

Aplicar a função para criar a coluna *semestre*.

In [4205]:
df_dados_filtrado['semestre'] = df_dados_filtrado.apply(
    lambda x: calcular_semestre(x['ano_ingresso'], x['ano']), axis=1
)

In [4206]:
# Agrupar por discente e disciplina e coletar os semestres
semestres_por_disciplina = df_dados_filtrado.groupby(['discente', 'nome'])['semestre'].apply(lambda x: ', '.join(map(str, x.unique()))).reset_index()

# Renomear a coluna para algo mais descritivo
semestres_por_disciplina.rename(columns={'semestre': 'semestres_cursados'}, inplace=True)

# Exibir o resultado
semestres_por_disciplina

,discente,nome,semestres_cursados
0,001cea3c82e2010681f2cdeab21e5ecf,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,1
1,001cea3c82e2010681f2cdeab21e5ecf,FUNDAMENTOS DE MATEMÁTICA,1
2,001cea3c82e2010681f2cdeab21e5ecf,INTRODUÇÃO À INFORMÁTICA,1
3,001cea3c82e2010681f2cdeab21e5ecf,LÓGICA,1
4,001cea3c82e2010681f2cdeab21e5ecf,TEORIA GERAL DA ADMINISTRAÇÃO,1
...,...,...,...
10701,ff56f2c5048dae0797fd3e851572b80c,TEORIA GERAL DOS SISTEMAS,2
10702,ff56f2c5048dae0797fd3e851572b80c,ÁLGEBRA LINEAR,8
10703,ffe447b2fe2058d45d5f756349a26f45,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,1
10704,ffe447b2fe2058d45d5f756349a26f45,INTRODUÇÃO À INFORMÁTICA,1


In [4207]:
# Supondo que tabela_final já existe e tem as colunas 'discente' e 'disciplina'
df_dados_filtrado = df_dados_filtrado.merge(semestres_por_disciplina, on=['discente' , 'nome'], how='left')
df_dados_filtrado 

,discente,unidade,nota,media_final,frequencia,descricao,ano,id_componente,nome,ch_total,sexo,ano_nascimento,ano_ingresso,status,classificacao,mulher,homem,ch_reprovacao,semestre,semestres_cursados
0,bad4e0ec61e635f382e7e32c32d5ad11,1.0,-1.0,-1.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0,1,1
1,bad4e0ec61e635f382e7e32c32d5ad11,2.0,-1.0,-1.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0,1,1
2,bad4e0ec61e635f382e7e32c32d5ad11,3.0,-1.0,-1.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1,1,0,330.0,1,1
3,bee926721a5b23a098fdfb850bf8d5b5,1.0,-1.0,-1.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1,0,1,330.0,1,1
4,bee926721a5b23a098fdfb850bf8d5b5,2.0,-1.0,-1.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1,0,1,330.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38085,22f4aed4a073c5e9515a8669e9c102f3,2.0,6.9,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1,0,1,240.0,8,"7, 8"
38086,22f4aed4a073c5e9515a8669e9c102f3,3.0,5.0,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1,0,1,240.0,8,"7, 8"
38087,943463d2de8c6a60ec5f5d959ba7f1ac,1.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2,0,1,60.0,10,10
38088,943463d2de8c6a60ec5f5d959ba7f1ac,2.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2,0,1,60.0,10,10


In [4208]:
semestres = df_dados_filtrado[['discente', 'semestre']].drop_duplicates("discente")
tabela_final = tabela_final.merge(semestres, on='discente', how='left')
tabela_final


,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,SISTEMAS OPERACIONAIS,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA,ano_ingresso,mulher,homem,status,classificacao,ch_cumprida,ch_cumprida_dividida,ch_reprovacao,taxa_reprovacao,semestre
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018,0,1,-1,-4,330,0.180328,NaN,NaN,1
1,005c14d7c07bf7980b60c703f99c5ee7,1.0,2.0,1.0,1.0,0.0,1.0,0.0,3.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2018,0,1,-1,-2,2160,1.180328,960.0,0.444444,1
2,0107fd69d8cd7e3d30dede96fb68bfe5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,2011,0,1,-1,-1,870,0.475410,270.0,0.310345,1
3,014789363f7940922e71e710ee9d22bc,2.0,3.0,1.0,0.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,1,-1,2310,1.262295,540.0,0.233766,1
4,014f0dec46fe7a9c5836527662e1df10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,0,1,-1,-3,240,0.131148,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,fe802d8d85de6f842749468401d1146c,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2022,0,1,0,-1,540,0.295082,270.0,0.500000,1
676,fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,1,2,2430,1.327869,480.0,0.197531,1
677,fec9ed6026d55ecdf514c640312c3d08,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2020,0,1,0,2,1350,0.737705,NaN,NaN,2
678,ff56f2c5048dae0797fd3e851572b80c,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2014,0,1,1,-1,2670,1.459016,630.0,0.235955,1


In [4209]:
df_dados_filtrado.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'frequencia', 'descricao',
       'ano', 'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status', 'classificacao', 'mulher', 'homem',
       'ch_reprovacao', 'semestre', 'semestres_cursados'],
      dtype='object')

### Coluna com o Total de **Semestre Cursado**

Identificar *semestres* únicos cursados por cada discente.

In [4210]:
semestres_unicos_por_discente = df_dados_filtrado.groupby('discente')['ano'].nunique().reset_index()
semestres_unicos_por_discente.rename(columns={'ano' : 'semestre_total'}, inplace=True)

Adicionando a quantidade de *semestres* cursados por cada discente.

In [4211]:
tabela_final = tabela_final.merge(semestres_unicos_por_discente, on='discente', how='left')

Dividindo a quantidade de *semestres cursados* pela quantidade de *semestres estimados* no curso.

In [4213]:
tabela_final['semestre_dividido'] = tabela_final['semestre_total'] / 8

### Coluna com o **Último Período** Matrículado

Encontrar o *último período* que cada aluno estudou no curso.

In [4214]:
ultimo_periodo = df_dados_filtrado.groupby('discente')['ano'].max().reset_index()
ultimo_periodo.rename(columns={'ano': 'ultimo_periodo'}, inplace=True)

Adicionando as informações do *último período* de cada discente.

In [4215]:
tabela_final = tabela_final.merge(ultimo_periodo, on='discente', how='left')

,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,SISTEMAS OPERACIONAIS,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA,ano_ingresso,mulher,homem,status,classificacao,ch_cumprida,ch_cumprida_dividida,ch_reprovacao,taxa_reprovacao,semestre,semestre_total,semestre_dividido,ultimo_periodo
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018,0,1,-1,-4,330,0.180328,NaN,NaN,1,1,0.125,20181
1,005c14d7c07bf7980b60c703f99c5ee7,1.0,2.0,1.0,1.0,0.0,1.0,0.0,3.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2018,0,1,-1,-2,2160,1.180328,960.0,0.444444,1,9,1.125,20221
2,0107fd69d8cd7e3d30dede96fb68bfe5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,2011,0,1,-1,-1,870,0.475410,270.0,0.310345,1,3,0.375,20121
3,014789363f7940922e71e710ee9d22bc,2.0,3.0,1.0,0.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,1,-1,2310,1.262295,540.0,0.233766,1,10,1.250,20206
4,014f0dec46fe7a9c5836527662e1df10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,0,1,-1,-3,240,0.131148,NaN,NaN,2,3,0.375,20206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,fe802d8d85de6f842749468401d1146c,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2022,0,1,0,-1,540,0.295082,270.0,0.500000,1,2,0.250,20222
676,fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,1,2,2430,1.327869,480.0,0.197531,1,11,1.375,20206
677,fec9ed6026d55ecdf514c640312c3d08,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2020,0,1,0,2,1350,0.737705,NaN,NaN,2,7,0.875,20222
678,ff56f2c5048dae0797fd3e851572b80c,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2014,0,1,1,-1,2670,1.459016,630.0,0.235955,1,12,1.500,20192


### Coluna com a quantidade de vezes que o discente se **Matriculou**

Soma dos valores das colunas para contabilizar a *matrícula* por discente.

In [4216]:
matricula =      tabela_final['ALGORITMOS E LÓGICA DE PROGRAMAÇÃO'] + \
                 tabela_final['INTRODUÇÃO À INFORMÁTICA'] + \
                 tabela_final['FUNDAMENTOS DE MATEMÁTICA'] + \
                 tabela_final['LÓGICA'] + \
                 tabela_final['TEORIA GERAL DA ADMINISTRAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO'] + \
                 tabela_final['CÁLCULO DIFERENCIAL E INTEGRAL'] + \
                 tabela_final['TEORIA GERAL DOS SISTEMAS'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS I'] + \
                 tabela_final['ESTRUTURA DE DADOS'] + \
                 tabela_final['ÁLGEBRA LINEAR'] + \
                 tabela_final['ORGANIZAÇÃO, SISTEMAS E MÉTODOS'] + \
                 tabela_final['FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO WEB'] + \
                 tabela_final['ARQUITETURA DE COMPUTADORES'] + \
                 tabela_final['PROBABILIDADE E ESTATÍSTICA'] + \
                 tabela_final['BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE I'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS II'] + \
                 tabela_final['SISTEMAS OPERACIONAIS'] + \
                 tabela_final['PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE II'] + \
                 tabela_final['REDES DE COMPUTADORES'] + \
                 tabela_final['EMPREENDEDORISMO EM INFORMÁTICA'] + \
                 tabela_final['GESTÃO DE PROJETO DE SOFTWARE'] + \
                 tabela_final['PROGRAMAÇÃO VISUAL'] + \
                 tabela_final['MATEMÁTICA FINANCEIRA'] + \
                 tabela_final['SISTEMAS DE APOIO À DECISÃO'] + \
                 tabela_final['ÉTICA']

Adicionando a coluna *matrícula* na tabela.

In [4217]:
tabela_final['matricula'] = matricula

In [4218]:
dados_da_coluna = tabela_final['matricula']
dados_da_coluna

0       5.0
1      33.0
2      13.0
3      36.0
4       4.0
       ... 
675     8.0
676    38.0
677    21.0
678    41.0
679     3.0
Name: matricula, Length: 680, dtype: float64

### Coluna com o **Tempo Relativo** que o discente passou no curso

Converter a coluna *ch_cumprida_dividida* para tipo numérico.

In [4219]:
tabela_final['semestre_dividido'] = pd.to_numeric(tabela_final['semestre_dividido'], errors='coerce')

Converter a coluna *status* para tipo numérico.

In [4220]:
tabela_final['status'] = pd.to_numeric(tabela_final['status'], errors='coerce')

Multiplicar os valores da coluna *ch_cumprida_dividida* pela coluna *status*.

In [4221]:
tabela_final['tempo_relativo'] = tabela_final['semestre_dividido'] * tabela_final['status']

### Coluna com a quantidade de vezes que o discente foi **Aprovado**

Filtrando dados pela unidade.

In [4222]:
filtrando_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

Listar todas as disciplinas.

In [4223]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *aprovações*.

In [4224]:
aprovacoes = filtrando_unidade[filtrando_unidade['descricao'].isin([
    'APROVADO',
    'APROVADO POR NOTA',
])]

Agrupar por discente e nome da disciplina para contar as *aprovações*.

In [4225]:
aprovacoes_count = aprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_APROVADO*.

In [4226]:
aprovacoes_count.columns = [f"{disciplina}_APROVADO" for disciplina in aprovacoes_count.columns]

Mesclar as *aprovações* de volta ao DataFrame original.

In [4227]:
# Criar `df_final` removendo duplicatas em 'discente' e juntando com `aprovacoes_count`
df_final = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(aprovacoes_count, on='discente').fillna(0).reset_index()

# Selecionar as colunas de df_final que não estão presentes em tabela_final
colunas_para_juntar = [col for col in df_final.columns if col not in tabela_final.columns or col == 'discente']

# Selecionar apenas as colunas necessárias
df_final_rename = df_final[colunas_para_juntar]

# Realizar a junção com tabela_final
tabela_final = tabela_final.merge(df_final_rename, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente foi **Reprovado**

Listar todas as disciplinas.

In [4228]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *reprovações* com o filtro da 1 unidade.

In [4229]:
reprovacoes = filtrando_unidade[filtrando_unidade['descricao'].isin([
    'REPROVADO',
    'REPROVADO POR NOTA',
    'REPROVADO POR FALTAS',
    'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR MÉDIA E POR FALTAS'
])]

Agrupar por discente e nome da disciplina para contar as *reprovações*.

In [4230]:
reprovacoes_count = reprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_REPROVADO*.

In [4231]:
reprovacoes_count.columns = [f"{disciplina}_REPROVADO" for disciplina in reprovacoes_count.columns]

Mesclar as *reprovações* de volta ao DataFrame original.

In [4232]:
# Criar `df_final` removendo duplicatas em 'discente' e juntando com `reprovacoes_count`
df_final = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(reprovacoes_count, on='discente').fillna(0).reset_index()

# Selecionar as colunas de df_final que não estão presentes em tabela_final
colunas_para_juntar = [col for col in df_final.columns if col not in tabela_final.columns or col == 'discente']

# Selecionar apenas as colunas necessárias
df_final_rename = df_final[colunas_para_juntar]

# Realizar a junção com tabela_final
tabela_final = tabela_final.merge(df_final_rename, on='discente', how='left')

## Pivoteamento

In [4233]:
df_pivot_f = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='frequencia')
df_pivot_f = df_pivot_f.add_suffix('_FALTAS')


In [4234]:

df_pivot_nota = df_dados_filtrado.pivot_table(index='discente', columns= ['nome','unidade'], values='nota', aggfunc='first')
df_pivot_nota.columns = df_pivot_nota.columns.set_levels(
    [level + '_NOTA_' for level in df_pivot_nota.columns.levels[0]],  # Aplicando o sufixo aos nomes do nível 'nome'
    level=0  # Indicando que estamos modificando o primeiro nível (nome)
)

In [4235]:
# Flatten the columns by joining different levels
df_pivot_nota.columns = [''.join(map(str, col)).strip() for col in df_pivot_nota.columns.values]

In [4236]:
df_pivot_mf = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='media_final',aggfunc='first')
df_pivot_mf = df_pivot_mf.add_suffix('_MEDIA_FINAL_1.0')

In [4237]:
df_pivot_mf_2 = df_dados_filtrado.pivot_table(index='discente', columns='nome', values='media_final', aggfunc='mean')
df_pivot_mf_2 = df_pivot_mf_2.add_suffix('_MEDIA_FINAL_2.0')

In [4238]:
# Criação da tabela pivô com o último valor de semestre_total
df_pivot_sem = filtrando_unidade.pivot_table(index='discente', 
                                                   columns='nome', 
                                                   values='semestre', 
                                                   aggfunc=lambda x: x.iloc[-1] if len(x) > 0 else np.nan)
df_pivot_sem

nome,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,SISTEMAS OPERACIONAIS,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA
discente,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
001cea3c82e2010681f2cdeab21e5ecf,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
005c14d7c07bf7980b60c703f99c5ee7,1.0,10.0,4.0,4.0,NaN,8.0,NaN,9.0,3.0,10.0,NaN,2.0,3.0,10.0,3.0,6.0,3.0,4.0,7.0,NaN,10.0,9.0,8.0,NaN,7.0,1.0,2.0,9.0,6.0
0107fd69d8cd7e3d30dede96fb68bfe5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN,3.0,3.0,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,3.0
014789363f7940922e71e710ee9d22bc,2.0,8.0,4.0,NaN,6.0,5.0,6.0,8.0,5.0,3.0,7.0,2.0,2.0,8.0,3.0,14.0,3.0,4.0,6.0,7.0,5.0,5.0,13.0,7.0,14.0,1.0,2.0,7.0,3.0
014f0dec46fe7a9c5836527662e1df10,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe802d8d85de6f842749468401d1146c,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN
fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,4.0,4.0,7.0,4.0,8.0,13.0,8.0,4.0,3.0,10.0,1.0,13.0,7.0,7.0,6.0,3.0,4.0,6.0,7.0,5.0,5.0,6.0,7.0,14.0,1.0,2.0,5.0,5.0
fec9ed6026d55ecdf514c640312c3d08,6.0,5.0,4.0,4.0,NaN,6.0,NaN,3.0,6.0,3.0,NaN,5.0,5.0,NaN,3.0,4.0,2.0,3.0,6.0,NaN,5.0,5.0,NaN,NaN,NaN,6.0,2.0,3.0,NaN


In [4239]:
# Agrupar por discente e disciplina e coletar os semestres
semestres_por_disciplina = filtrando_unidade.groupby(['discente', 'nome'])['semestre_total'].apply(lambda x: ', '.join(map(str, x))).reset_index()
df_pivot_sem = semestres_por_disciplina.pivot_table(index='discente', columns='nome', values='semestre_total', aggfunc='first')
df_pivot_sem = df_pivot_sem.add_suffix('_SEMESTRE')
df_pivot_sem

nome,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_SEMESTRE,ARQUITETURA DE COMPUTADORES_SEMESTRE,BANCO DE DADOS_SEMESTRE,CÁLCULO DIFERENCIAL E INTEGRAL_SEMESTRE,EMPREENDEDORISMO EM INFORMÁTICA_SEMESTRE,ENGENHARIA DE SOFTWARE I_SEMESTRE,ENGENHARIA DE SOFTWARE II_SEMESTRE,ESTRUTURA DE DADOS_SEMESTRE,FUNDAMENTOS DE MATEMÁTICA_SEMESTRE,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_SEMESTRE,GESTÃO DE PROJETO DE SOFTWARE_SEMESTRE,INTRODUÇÃO À INFORMÁTICA_SEMESTRE,LÓGICA_SEMESTRE,MATEMÁTICA FINANCEIRA_SEMESTRE,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS_SEMESTRE",PROBABILIDADE E ESTATÍSTICA_SEMESTRE,PROGRAMAÇÃO_SEMESTRE,PROGRAMAÇÃO ORIENTADA A OBJETOS I_SEMESTRE,PROGRAMAÇÃO ORIENTADA A OBJETOS II_SEMESTRE,PROGRAMAÇÃO VISUAL_SEMESTRE,PROGRAMAÇÃO WEB_SEMESTRE,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_SEMESTRE,REDES DE COMPUTADORES_SEMESTRE,SISTEMAS DE APOIO À DECISÃO_SEMESTRE,SISTEMAS OPERACIONAIS_SEMESTRE,TEORIA GERAL DA ADMINISTRAÇÃO_SEMESTRE,TEORIA GERAL DOS SISTEMAS_SEMESTRE,ÁLGEBRA LINEAR_SEMESTRE,ÉTICA_SEMESTRE
discente,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
001cea3c82e2010681f2cdeab21e5ecf,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
005c14d7c07bf7980b60c703f99c5ee7,9,"9, 9",9,9,NaN,9,NaN,"9, 9, 9","9, 9",9,NaN,"9, 9","9, 9",9,9,9,"9, 9",9,9,NaN,9,"9, 9",9,NaN,9,9,9,"9, 9",9
0107fd69d8cd7e3d30dede96fb68bfe5,"3, 3",NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,NaN,"3, 3","3, 3",NaN,3,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,NaN,3
014789363f7940922e71e710ee9d22bc,"10, 10","10, 10, 10",10,NaN,10,10,10,"10, 10","10, 10, 10",10,10,"10, 10","10, 10",10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
014f0dec46fe7a9c5836527662e1df10,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe802d8d85de6f842749468401d1146c,"2, 2",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2, 2",NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN
fe87dfa176a74fc10a5cb701b9fb5dd4,11,"11, 11",11,"11, 11",11,11,11,"11, 11","11, 11, 11",11,11,11,"11, 11, 11",11,"11, 11",11,"11, 11",11,11,11,11,11,11,11,11,11,11,11,11
fec9ed6026d55ecdf514c640312c3d08,7,7,7,7,NaN,7,NaN,7,7,7,NaN,7,7,NaN,7,7,7,7,7,NaN,"7, 7",7,NaN,NaN,NaN,7,7,7,NaN


In [4240]:
df_merged_1 = pd.merge(df_pivot_f, df_pivot_nota, on='discente')
df_merged_2 = pd.merge(df_merged_1, df_pivot_mf, on='discente')
df_merged_3 = pd.merge(df_merged_2, df_pivot_mf_2, on='discente')
df_merged_4 = pd.merge(df_merged_3, df_pivot_sem, on='discente')
df_merged_5 = pd.merge(df_merged_4, tabela_final, on='discente')


Substituir todos os valores 0 por -1.

In [4241]:
df_merged_4.replace(0, -1, inplace=True)
df_merged_4

,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS,ARQUITETURA DE COMPUTADORES_FALTAS,BANCO DE DADOS_FALTAS,CÁLCULO DIFERENCIAL E INTEGRAL_FALTAS,EMPREENDEDORISMO EM INFORMÁTICA_FALTAS,ENGENHARIA DE SOFTWARE I_FALTAS,ENGENHARIA DE SOFTWARE II_FALTAS,ESTRUTURA DE DADOS_FALTAS,FUNDAMENTOS DE MATEMÁTICA_FALTAS,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_FALTAS,GESTÃO DE PROJETO DE SOFTWARE_FALTAS,INTRODUÇÃO À INFORMÁTICA_FALTAS,LÓGICA_FALTAS,MATEMÁTICA FINANCEIRA_FALTAS,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS_FALTAS",PROBABILIDADE E ESTATÍSTICA_FALTAS,PROGRAMAÇÃO_FALTAS,PROGRAMAÇÃO ORIENTADA A OBJETOS I_FALTAS,PROGRAMAÇÃO ORIENTADA A OBJETOS II_FALTAS,PROGRAMAÇÃO VISUAL_FALTAS,PROGRAMAÇÃO WEB_FALTAS,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_FALTAS,REDES DE COMPUTADORES_FALTAS,SISTEMAS DE APOIO À DECISÃO_FALTAS,SISTEMAS OPERACIONAIS_FALTAS,TEORIA GERAL DA ADMINISTRAÇÃO_FALTAS,TEORIA GERAL DOS SISTEMAS_FALTAS,ÁLGEBRA LINEAR_FALTAS,ÉTICA_FALTAS,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0,ARQUITETURA DE COMPUTADORES_NOTA_1.0,ARQUITETURA DE COMPUTADORES_NOTA_2.0,ARQUITETURA DE COMPUTADORES_NOTA_3.0,BANCO DE DADOS_NOTA_1.0,BANCO DE DADOS_NOTA_2.0,BANCO DE DADOS_NOTA_3.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_1.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_2.0,CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_3.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_1.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_2.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_3.0,ENGENHARIA DE SOFTWARE I_NOTA_1.0,ENGENHARIA DE SOFTWARE I_NOTA_2.0,ENGENHARIA DE SOFTWARE I_NOTA_3.0,ENGENHARIA DE SOFTWARE II_NOTA_1.0,ENGENHARIA DE SOFTWARE II_NOTA_2.0,ENGENHARIA DE SOFTWARE II_NOTA_3.0,ESTRUTURA DE DADOS_NOTA_1.0,ESTRUTURA DE DADOS_NOTA_2.0,ESTRUTURA DE DADOS_NOTA_3.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_1.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_2.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_3.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_1.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_2.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_3.0,INTRODUÇÃO À INFORMÁTICA_NOTA_1.0,INTRODUÇÃO À INFORMÁTICA_NOTA_2.0,INTRODUÇÃO À INFORMÁTICA_NOTA_3.0,...,REDES DE COMPUTADORES_MEDIA_FINAL_1.0,SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_1.0,SISTEMAS OPERACIONAIS_MEDIA_FINAL_1.0,TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_1.0,TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_1.0,ÁLGEBRA LINEAR_MEDIA_FINAL_1.0,ÉTICA_MEDIA_FINAL_1.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2.0,ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_2.0,BANCO DE DADOS_MEDIA_FINAL_2.0,CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_2.0,EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_2.0,ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_2.0,ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_2.0,ESTRUTURA DE DADOS_MEDIA_FINAL_2.0,FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_2.0,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_2.0,GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_2.0,INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2.0,LÓGICA_MEDIA_FINAL_2.0,MATEMÁTICA FINANCEIRA_MEDIA_FINAL_2.0,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_2.0",PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_2.0,PROGRAMAÇÃO_MEDIA_FINAL_2.0,PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_2.0,PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_2.0,PROGRAMAÇÃO VISUAL_MEDIA_FINAL_2.0,PROGRAMAÇÃO WEB_MEDIA_FINAL_2.0,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_2.0,REDES DE COMPUTADORES_MEDIA_FINAL_2.0,SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_2.0,SISTEMAS OPERACIONAIS_MEDIA_FINAL_2.0,TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2.0,TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_2.0,ÁLGEBRA LINEAR_MEDIA_FINAL_2.0,ÉTICA_MEDIA_FINAL_2.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_SEMESTRE,ARQUITETURA DE COMPUTADORES_SEMESTRE,BANCO DE DADOS_SEMESTRE,CÁLCULO DIFERENCIAL E INTEGRAL_SEMESTRE,EMPREENDEDORISMO EM INFORMÁTICA_SEMESTRE,ENGENHARIA DE SOFTWARE I_SEMESTRE,ENGENHARIA DE SOFTWARE II_SEMESTRE,ESTRUTURA DE DADOS_SEMESTRE,FUNDAMENTOS DE

Substituindo NaN por 0 nas colunas.

In [4242]:
df_merged_5 = df_merged_5.fillna(0)


Arredondando todas as colunas para 1 casa decimal.

In [4243]:
df_merged_5 = df_merged_5.round(2)

Mudando a ordem das colunas.

In [4244]:
colunas_ordenadas = ['homem', 'mulher', 'ch_cumprida',
                    'ch_cumprida_dividida', 'ch_reprovacao',
                    'taxa_reprovacao', 'semestre_total', 'semestre_dividido', 'matricula',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_SEMESTRE',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1.0',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2.0',
                    'INTRODUÇÃO À INFORMÁTICA',
                    'INTRODUÇÃO À INFORMÁTICA_SEMESTRE',
                    'INTRODUÇÃO À INFORMÁTICA_APROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_REPROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_FALTAS',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_1.0',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_2.0',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_3.0',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_1.0',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2.0',
                    'FUNDAMENTOS DE MATEMÁTICA',
                    'FUNDAMENTOS DE MATEMÁTICA_SEMESTRE',
                    'FUNDAMENTOS DE MATEMÁTICA_APROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_REPROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_FALTAS',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0',
                    'FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0',
                    'FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_1.0',
                    'FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_2.0',
                    'LÓGICA',
                    'LÓGICA_SEMESTRE',
                    'LÓGICA_APROVADO',
                    'LÓGICA_REPROVADO',
                    'LÓGICA_FALTAS',
                    'LÓGICA_NOTA_1.0',
                    'LÓGICA_NOTA_2.0',
                    'LÓGICA_NOTA_3.0',
                    'LÓGICA_MEDIA_FINAL_1.0',
                    'LÓGICA_MEDIA_FINAL_2.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_SEMESTRE',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_FALTAS',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_1.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_2.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_3.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_1.0',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO',
                    'PROGRAMAÇÃO_SEMESTRE',
                    'PROGRAMAÇÃO_APROVADO',
                    'PROGRAMAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO_FALTAS',
                    'PROGRAMAÇÃO_NOTA_1.0',
                    'PROGRAMAÇÃO_NOTA_2.0',
                    'PROGRAMAÇÃO_NOTA_3.0',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_SEMESTRE',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_APROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_REPROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_FALTAS',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_1.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_2.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_NOTA_3.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_1.0',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_MEDIA_FINAL_2.0',
                    'TEORIA GERAL DOS SISTEMAS',
                    'TEORIA GERAL DOS SISTEMAS_SEMESTRE',
                    'TEORIA GERAL DOS SISTEMAS_APROVADO',
                    'TEORIA GERAL DOS SISTEMAS_REPROVADO',
                    'TEORIA GERAL DOS SISTEMAS_FALTAS',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_1.0',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_2.0',
                    'TEORIA GERAL DOS SISTEMAS_NOTA_3.0',
                    'TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_1.0',
                    'TEORIA GERAL DOS SISTEMAS_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_SEMESTRE',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_FALTAS',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_NOTA_3.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_MEDIA_FINAL_2.0',
                    'ESTRUTURA DE DADOS',
                    'ESTRUTURA DE DADOS_SEMESTRE',
                    'ESTRUTURA DE DADOS_APROVADO',
                    'ESTRUTURA DE DADOS_REPROVADO',
                    'ESTRUTURA DE DADOS_FALTAS',
                    'ESTRUTURA DE DADOS_NOTA_1.0',
                    'ESTRUTURA DE DADOS_NOTA_2.0',
                    'ESTRUTURA DE DADOS_NOTA_3.0',
                    'ESTRUTURA DE DADOS_MEDIA_FINAL_1.0',
                    'ESTRUTURA DE DADOS_MEDIA_FINAL_2.0',
                    'ÁLGEBRA LINEAR',
                    'ÁLGEBRA LINEAR_SEMESTRE',
                    'ÁLGEBRA LINEAR_APROVADO',
                    'ÁLGEBRA LINEAR_REPROVADO',
                    'ÁLGEBRA LINEAR_FALTAS',
                    'ÁLGEBRA LINEAR_NOTA_1.0',
                    'ÁLGEBRA LINEAR_NOTA_2.0',
                    'ÁLGEBRA LINEAR_NOTA_3.0',
                    'ÁLGEBRA LINEAR_MEDIA_FINAL_1.0',
                    'ÁLGEBRA LINEAR_MEDIA_FINAL_2.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_SEMESTRE',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_APROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_FALTAS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_1.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_2.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_NOTA_3.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_1.0',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_MEDIA_FINAL_2.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_SEMESTRE',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_APROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_FALTAS',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_1.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_2.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_NOTA_3.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_1.0',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO WEB',
                    'PROGRAMAÇÃO WEB_SEMESTRE',
                    'PROGRAMAÇÃO WEB_APROVADO',
                    'PROGRAMAÇÃO WEB_REPROVADO',
                    'PROGRAMAÇÃO WEB_FALTAS',
                    'PROGRAMAÇÃO WEB_NOTA_1.0',
                    'PROGRAMAÇÃO WEB_NOTA_2.0',
                    'PROGRAMAÇÃO WEB_NOTA_3.0',
                    'PROGRAMAÇÃO WEB_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO WEB_MEDIA_FINAL_2.0',
                    'ARQUITETURA DE COMPUTADORES',
                    'ARQUITETURA DE COMPUTADORES_SEMESTRE',
                    'ARQUITETURA DE COMPUTADORES_APROVADO',
                    'ARQUITETURA DE COMPUTADORES_REPROVADO',
                    'ARQUITETURA DE COMPUTADORES_FALTAS',
                    'ARQUITETURA DE COMPUTADORES_NOTA_1.0',
                    'ARQUITETURA DE COMPUTADORES_NOTA_2.0',
                    'ARQUITETURA DE COMPUTADORES_NOTA_3.0',
                    'ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_1.0',
                    'ARQUITETURA DE COMPUTADORES_MEDIA_FINAL_2.0',
                    'PROBABILIDADE E ESTATÍSTICA',
                    'PROBABILIDADE E ESTATÍSTICA_SEMESTRE',
                    'PROBABILIDADE E ESTATÍSTICA_APROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_REPROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_FALTAS',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_1.0',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_2.0',
                    'PROBABILIDADE E ESTATÍSTICA_NOTA_3.0',
                    'PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_1.0',
                    'PROBABILIDADE E ESTATÍSTICA_MEDIA_FINAL_2.0',
                    'BANCO DE DADOS',
                    'BANCO DE DADOS_SEMESTRE',
                    'BANCO DE DADOS_APROVADO',
                    'BANCO DE DADOS_REPROVADO',
                    'BANCO DE DADOS_FALTAS',
                    'BANCO DE DADOS_NOTA_1.0',
                    'BANCO DE DADOS_NOTA_2.0',
                    'BANCO DE DADOS_NOTA_3.0',
                    'BANCO DE DADOS_MEDIA_FINAL_1.0',
                    'BANCO DE DADOS_MEDIA_FINAL_2.0',
                    'ENGENHARIA DE SOFTWARE I',
                    'ENGENHARIA DE SOFTWARE I_SEMESTRE',
                    'ENGENHARIA DE SOFTWARE I_APROVADO',
                    'ENGENHARIA DE SOFTWARE I_REPROVADO',
                    'ENGENHARIA DE SOFTWARE I_FALTAS',
                    'ENGENHARIA DE SOFTWARE I_NOTA_1.0',
                    'ENGENHARIA DE SOFTWARE I_NOTA_2.0',
                    'ENGENHARIA DE SOFTWARE I_NOTA_3.0',
                    'ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_1.0',
                    'ENGENHARIA DE SOFTWARE I_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_SEMESTRE',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_FALTAS',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_2.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_NOTA_3.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_MEDIA_FINAL_2.0',
                    'SISTEMAS OPERACIONAIS',
                    'SISTEMAS OPERACIONAIS_SEMESTRE',
                    'SISTEMAS OPERACIONAIS_APROVADO',
                    'SISTEMAS OPERACIONAIS_REPROVADO',
                    'SISTEMAS OPERACIONAIS_FALTAS',
                    'SISTEMAS OPERACIONAIS_NOTA_1.0',
                    'SISTEMAS OPERACIONAIS_NOTA_2.0',
                    'SISTEMAS OPERACIONAIS_NOTA_3.0',
                    'SISTEMAS OPERACIONAIS_MEDIA_FINAL_1.0',
                    'SISTEMAS OPERACIONAIS_MEDIA_FINAL_2.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_SEMESTRE',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_APROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_FALTAS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_1.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_2.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_NOTA_3.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_1.0',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_MEDIA_FINAL_2.0',
                    'ENGENHARIA DE SOFTWARE II',
                    'ENGENHARIA DE SOFTWARE II_SEMESTRE',
                    'ENGENHARIA DE SOFTWARE II_APROVADO',
                    'ENGENHARIA DE SOFTWARE II_REPROVADO',
                    'ENGENHARIA DE SOFTWARE II_FALTAS',
                    'ENGENHARIA DE SOFTWARE II_NOTA_1.0',
                    'ENGENHARIA DE SOFTWARE II_NOTA_2.0',
                    'ENGENHARIA DE SOFTWARE II_NOTA_3.0',
                    'ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_1.0',
                    'ENGENHARIA DE SOFTWARE II_MEDIA_FINAL_2.0',
                    'REDES DE COMPUTADORES',
                    'REDES DE COMPUTADORES_SEMESTRE',
                    'REDES DE COMPUTADORES_APROVADO',
                    'REDES DE COMPUTADORES_REPROVADO',
                    'REDES DE COMPUTADORES_FALTAS',
                    'REDES DE COMPUTADORES_NOTA_1.0',
                    'REDES DE COMPUTADORES_NOTA_2.0',
                    'REDES DE COMPUTADORES_NOTA_3.0',
                    'REDES DE COMPUTADORES_MEDIA_FINAL_1.0',
                    'REDES DE COMPUTADORES_MEDIA_FINAL_2.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA',
                    'EMPREENDEDORISMO EM INFORMÁTICA_SEMESTRE',
                    'EMPREENDEDORISMO EM INFORMÁTICA_APROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_REPROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_FALTAS',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_1.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_2.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_NOTA_3.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_1.0',
                    'EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_2.0',
                    'GESTÃO DE PROJETO DE SOFTWARE',
                    'GESTÃO DE PROJETO DE SOFTWARE_SEMESTRE',
                    'GESTÃO DE PROJETO DE SOFTWARE_APROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_REPROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_FALTAS',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_1.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_2.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_NOTA_3.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_1.0',
                    'GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_2.0',
                    'PROGRAMAÇÃO VISUAL',
                    'PROGRAMAÇÃO VISUAL_SEMESTRE',
                    'PROGRAMAÇÃO VISUAL_APROVADO',
                    'PROGRAMAÇÃO VISUAL_REPROVADO',
                    'PROGRAMAÇÃO VISUAL_FALTAS',
                    'PROGRAMAÇÃO VISUAL_NOTA_1.0',
                    'PROGRAMAÇÃO VISUAL_NOTA_2.0',
                    'PROGRAMAÇÃO VISUAL_NOTA_3.0',
                    'PROGRAMAÇÃO VISUAL_MEDIA_FINAL_1.0',
                    'PROGRAMAÇÃO VISUAL_MEDIA_FINAL_2.0',
                    'MATEMÁTICA FINANCEIRA',
                    'MATEMÁTICA FINANCEIRA_SEMESTRE',
                    'MATEMÁTICA FINANCEIRA_APROVADO',
                    'MATEMÁTICA FINANCEIRA_REPROVADO',
                    'MATEMÁTICA FINANCEIRA_FALTAS',
                    'MATEMÁTICA FINANCEIRA_NOTA_1.0',
                    'MATEMÁTICA FINANCEIRA_NOTA_2.0',
                    'MATEMÁTICA FINANCEIRA_NOTA_3.0',
                    'MATEMÁTICA FINANCEIRA_MEDIA_FINAL_1.0',
                    'MATEMÁTICA FINANCEIRA_MEDIA_FINAL_2.0',
                    'SISTEMAS DE APOIO À DECISÃO',
                    'SISTEMAS DE APOIO À DECISÃO_SEMESTRE',
                    'SISTEMAS DE APOIO À DECISÃO_APROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_REPROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_FALTAS',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_1.0',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_2.0',
                    'SISTEMAS DE APOIO À DECISÃO_NOTA_3.0',
                    'SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_1.0',
                    'SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_2.0',
                    'ÉTICA',
                    'ÉTICA_SEMESTRE',
                    'ÉTICA_APROVADO',
                    'ÉTICA_REPROVADO',
                    'ÉTICA_FALTAS',
                    'ÉTICA_NOTA_1.0',
                    'ÉTICA_NOTA_2.0',
                    'ÉTICA_NOTA_3.0',
                    'ÉTICA_MEDIA_FINAL_1.0',
                    'ÉTICA_MEDIA_FINAL_2.0',
                    'tempo_relativo', 'status']


Reordenando as colunas.

In [4245]:
df_merged_5 = df_merged_5[colunas_ordenadas]
df_merged_5

,homem,mulher,ch_cumprida,ch_cumprida_dividida,ch_reprovacao,taxa_reprovacao,semestre_total,semestre_dividido,matricula,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_SEMESTRE,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FALTAS,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1.0,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2.0,INTRODUÇÃO À INFORMÁTICA,INTRODUÇÃO À INFORMÁTICA_SEMESTRE,INTRODUÇÃO À INFORMÁTICA_APROVADO,INTRODUÇÃO À INFORMÁTICA_REPROVADO,INTRODUÇÃO À INFORMÁTICA_FALTAS,INTRODUÇÃO À INFORMÁTICA_NOTA_1.0,INTRODUÇÃO À INFORMÁTICA_NOTA_2.0,INTRODUÇÃO À INFORMÁTICA_NOTA_3.0,INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_1.0,INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2.0,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE MATEMÁTICA_SEMESTRE,FUNDAMENTOS DE MATEMÁTICA_APROVADO,FUNDAMENTOS DE MATEMÁTICA_REPROVADO,FUNDAMENTOS DE MATEMÁTICA_FALTAS,FUNDAMENTOS DE MATEMÁTICA_NOTA_1.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_2.0,FUNDAMENTOS DE MATEMÁTICA_NOTA_3.0,FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_1.0,FUNDAMENTOS DE MATEMÁTICA_MEDIA_FINAL_2.0,LÓGICA,LÓGICA_SEMESTRE,LÓGICA_APROVADO,LÓGICA_REPROVADO,LÓGICA_FALTAS,LÓGICA_NOTA_1.0,LÓGICA_NOTA_2.0,LÓGICA_NOTA_3.0,LÓGICA_MEDIA_FINAL_1.0,LÓGICA_MEDIA_FINAL_2.0,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DA ADMINISTRAÇÃO_SEMESTRE,TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO,TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO,TEORIA GERAL DA ADMINISTRAÇÃO_FALTAS,TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_1.0,TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_2.0,TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_3.0,TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_1.0,TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2.0,PROGRAMAÇÃO,PROGRAMAÇÃO_SEMESTRE,PROGRAMAÇÃO_APROVADO,PROGRAMAÇÃO_REPROVADO,PROGRAMAÇÃO_FALTAS,PROGRAMAÇÃO_NOTA_1.0,...,REDES DE COMPUTADORES_NOTA_3.0,REDES DE COMPUTADORES_MEDIA_FINAL_1.0,REDES DE COMPUTADORES_MEDIA_FINAL_2.0,EMPREENDEDORISMO EM INFORMÁTICA,EMPREENDEDORISMO EM INFORMÁTICA_SEMESTRE,EMPREENDEDORISMO EM INFORMÁTICA_APROVADO,EMPREENDEDORISMO EM INFORMÁTICA_REPROVADO,EMPREENDEDORISMO EM INFORMÁTICA_FALTAS,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_1.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_2.0,EMPREENDEDORISMO EM INFORMÁTICA_NOTA_3.0,EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_1.0,EMPREENDEDORISMO EM INFORMÁTICA_MEDIA_FINAL_2.0,GESTÃO DE PROJETO DE SOFTWARE,GESTÃO DE PROJETO DE SOFTWARE_SEMESTRE,GESTÃO DE PROJETO DE SOFTWARE_APROVADO,GESTÃO DE PROJETO DE SOFTWARE_REPROVADO,GESTÃO DE PROJETO DE SOFTWARE_FALTAS,GESTÃO DE PROJETO DE SOFTWARE_NOTA_1.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_2.0,GESTÃO DE PROJETO DE SOFTWARE_NOTA_3.0,GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_1.0,GESTÃO DE PROJETO DE SOFTWARE_MEDIA_FINAL_2.0,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO VISUAL_SEMESTRE,PROGRAMAÇÃO VISUAL_APROVADO,PROGRAMAÇÃO VISUAL_REPROVADO,PROGRAMAÇÃO VISUAL_FALTAS,PROGRAMAÇÃO VISUAL_NOTA_1.0,PROGRAMAÇÃO VISUAL_NOTA_2.0,PROGRAMAÇÃO VISUAL_NOTA_3.0,PROGRAMAÇÃO VISUAL_MEDIA_FINAL_1.0,PROGRAMAÇÃO VISUAL_MEDIA_FINAL_2.0,MATEMÁTICA FINANCEIRA,MATEMÁTICA FINANCEIRA_SEMESTRE,MATEMÁTICA FINANCEIRA_APROVADO,MATEMÁTICA FINANCEIRA_REPROVADO,MATEMÁTICA FINANCEIRA_FALTAS,MATEMÁTICA FINANCEIRA_NOTA_1.0,MATEMÁTICA FINANCEIRA_NOTA_2.0,MATEMÁTICA FINANCEIRA_NOTA_3.0,MATEMÁTICA FINANCEIRA_MEDIA_FINAL_1.0,MATEMÁTICA FINANCEIRA_MEDIA_FINAL_2.0,SISTEMAS DE APOIO À DECISÃO,SISTEMAS DE APOIO À DECISÃO_SEMESTRE,SISTEMAS DE APOIO À DECISÃO_APROVADO,SISTEMAS DE APOIO À DECISÃO_REPROVADO,SISTEMAS DE APOIO À DECISÃO_FALTAS,SISTEMAS DE APOIO À DECISÃO_NOTA_1.0,SISTEMAS DE APOIO À DECISÃO_NOTA_2.0,SISTEMAS DE APOIO À DECISÃO_NOTA_3.0,SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_1.0,SISTEMAS DE APOIO À DECISÃO_MEDIA_FINAL_2.0,ÉTICA,ÉTICA_SEMESTRE,ÉTICA_APROVADO,ÉTICA_REPROVADO,ÉTICA_FALTAS,ÉTICA_NOTA_1.0,ÉTICA_NOTA_2.0,ÉTICA_NOTA_3.0,ÉTICA_MEDIA_FINAL_1.0,ÉTICA_MEDIA_FINAL_2.0,tempo_relativo,stat

Para salvar o DataFrame no formato CSV com a separação por ';' e garantir que os dados numéricos estejam no formato correto.

In [4246]:
df_merged_5.to_csv('df_merged_5', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [4247]:
df_dados_filtrado.to_csv('df_filtrado.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [4248]:
tabela_final.to_csv('tabela_final.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)